## Load Libraries

In [10]:
import cv2
import pyaudio
import threading
import os
from datetime import datetime
import wave
import subprocess

#### Intelligent Interview Recording System
#### This module provides a robust screen recording solution with audio-video capture 
#### and merging capabilities, designed for interview analysis and recording.

### Key Features:
- Simultaneous video and audio recording
- Unique filename generation with timestamps
- Automatic file merging using FFmpeg
- Configurable output directory
- Error handling for camera and audio device access

#### Class for recording functions to create a simplified version that starts and stops recording when specific buttons are pressed

In [ ]:

class ScreenRecorder:
    """
    A comprehensive screen recording class that captures video and audio 
    simultaneously and merges them into a single output file.

    Attributes:
        output_dir (str): Directory to save recorded files
        FORMAT (int): Audio format (16-bit integer)
        CHANNELS (int): Number of audio channels (mono)
        RATE (int): Audio sample rate
        CHUNK (int): Audio buffer size
    """

    def __init__(self, output_dir='recordings'):
        """
        Initialize the ScreenRecorder with configuration settings.

        Args:
            output_dir (str, optional): Directory to save recordings. 
                                        Defaults to 'recordings'.
        """
        # Ensure output directory exists
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

        # Audio recording configuration
        self.FORMAT = pyaudio.paInt16  # 16-bit audio
        self.CHANNELS = 1  # Mono audio
        self.RATE = 44100  # Standard audio sample rate
        self.CHUNK = 1024  # Audio buffer size

        # File path placeholders
        self.video_path = None      # Raw video file path
        self.audio_path = None      # Raw audio file path
        self.final_output_path = None  # Merged video file path

        # Recording state management
        self.recording = False
        self.capture = None
        self.video_writer = None
        self.audio_stream = None
        self.audio_frames = []

    def start_recording(self):
        """
        Initiate screen recording process.

        Responsibilities:
        - Check if recording is already in progress
        - Generate unique filenames
        - Initialize video capture
        - Initialize audio capture
        - Start recording threads

        Raises:
            SystemError: If camera cannot be accessed
        """
        # Prevent multiple recording sessions
        if self.recording:
            print("Recording is already in progress.")
            return
        
        # Generate unique timestamps for filenames
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.video_path = os.path.join(self.output_dir, f"video_{timestamp}.avi")
        self.audio_path = os.path.join(self.output_dir, f"audio_{timestamp}.wav")
        self.final_output_path = os.path.join(self.output_dir, f"final_output_{timestamp}.mp4")

        # Initialize video capture
        self.capture = cv2.VideoCapture(0)
        if not self.capture.isOpened():
            print("Error: Unable to access the camera.")
            return

        # Retrieve video parameters
        frame_width = int(self.capture.get(3))   # Camera width
        frame_height = int(self.capture.get(4))  # Camera height
        frame_rate = int(self.capture.get(5)) or 30  # Frame rate or default

        # Create video writer object
        self.video_writer = cv2.VideoWriter(
            self.video_path,
            cv2.VideoWriter_fourcc(*"XVID"),  # Video codec
            frame_rate,
            (frame_width, frame_height)
        )

        # Initialize audio recording
        self.audio_frames = []
        self.audio_stream = pyaudio.PyAudio().open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            frames_per_buffer=self.CHUNK
        )

        # Start recording threads
        self.recording = True
        threading.Thread(target=self._record_video, daemon=True).start()
        threading.Thread(target=self._record_audio, daemon=True).start()

        print(f"Recording started. Video will be saved to: {self.video_path}")
        print(f"Audio will be saved to: {self.audio_path}")

    def _record_video(self):
        """
        Internal method to continuously capture video frames.
        Runs in a separate thread during recording.
        """
        while self.recording:
            ret, frame = self.capture.read()
            if not ret:
                print("Error: Unable to read from camera.")
                break
            self.video_writer.write(frame)

    def _record_audio(self):
        """
        Internal method to continuously capture audio frames.
        Runs in a separate thread during recording.
        """
        while self.recording:
            data = self.audio_stream.read(self.CHUNK)
            self.audio_frames.append(data)

    def stop_recording(self):
        """
        Stop the recording process and merge audio-video files.

        Returns:
            tuple: Paths for video, audio, and final merged file
        """
        # Prevent stopping non-existent recording
        if not self.recording:
            print("No recording in progress.")
            return None, None, None

        # Stop recording
        self.recording = False

        # Release system resources
        self.capture.release()
        self.video_writer.release()

        # Save audio file
        with wave.open(self.audio_path, 'wb') as wf:
            wf.setnchannels(self.CHANNELS)
            wf.setsampwidth(pyaudio.PyAudio().get_sample_size(self.FORMAT))
            wf.setframerate(self.RATE)
            wf.writeframes(b''.join(self.audio_frames))

        # Merge video and audio
        self._merge_video_audio()

        # Output recording details
        print(f"Recording stopped.")
        print(f"Video path: {self.video_path}")
        print(f"Audio path: {self.audio_path}")
        print(f"Final output path: {self.final_output_path}")

        return self.video_path, self.audio_path, self.final_output_path

    def _merge_video_audio(self):
        """
        Merge video and audio files using FFmpeg.

        Handles:
        - Video and audio merging
        - Error checking for FFmpeg availability
        - Codec conversion

        Raises:
            subprocess.CalledProcessError: FFmpeg command execution error
            FileNotFoundError: FFmpeg not installed
        """
        try:
            # FFmpeg command for merging files
            merge_command = [
                'ffmpeg',
                '-i', self.video_path,      # Input video
                '-i', self.audio_path,      # Input audio
                '-c:v', 'copy',             # Copy video codec
                '-c:a', 'aac',              # Convert audio to AAC
                '-shortest',                # Match shorter file duration
                self.final_output_path      # Output merged file
            ]
            
            # Execute FFmpeg merge
            subprocess.run(merge_command, check=True)
            print("Successfully merged video and audio")
        
        except subprocess.CalledProcessError as e:
            print(f"Error merging video and audio: {e}")
        except FileNotFoundError:
            print("FFmpeg not found. Please install FFmpeg to merge video and audio.")


In [14]:
recorder = ScreenRecorder()
input("Press Enter to start recording...")
recorder.start_recording()
    
input("Press Enter to stop recording...")
video_path, audio_path, final_output_path = recorder.stop_recording()

Recording started. Video will be saved to: recordings\video_20241203_225910.avi
Audio will be saved to: recordings\audio_20241203_225910.wav
Successfully merged video and audio
Recording stopped.
Video path: recordings\video_20241203_225910.avi
Audio path: recordings\audio_20241203_225910.wav
Final output path: recordings\final_output_20241203_225910.mp4


In [18]:
print(video_path,"  ", audio_path, "     ",final_output_path)

recordings\video_20241203_225910.avi    recordings\audio_20241203_225910.wav       recordings\final_output_20241203_225910.mp4
